In [ ]:
import pandas as pd
import dateutil.parser as dp
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib
import os
import glob
from collections import OrderedDict
import numpy as np
import scipy

In [ ]:
frameworks = ['safire', 'halucinator', 'libafl_halucinator', 'fuzzware_hal', 'fuzzware_nohal',  ]

In [ ]:
targets = ['atmel_6lowpan_udp_rx', 'atmel_6lowpan_udp_tx', 'nxp_lwip_http', 'samr21_http', 'p2im_plc',  'p2im_drone', 'st-plc', 'wycinwyc', 'stm32_tcp_echo_client', 'stm32_tcp_echo_server', 'stm32_udp_echo_client', 'stm32_udp_echo_server']

In [ ]:
def remove_duplicates(data):
    data_in = OrderedDict(data)
    data_out = {}
    for k, v in data_in.items():
        if v not in data_out.values():
            data_out[k] = v
    return OrderedDict(data_out)

def parse_fuzzware_data(data):
    data = data.split('\n')[1:]
    data = [(d.split('\t')[:2]) for d in data if d != '']
    data = {int(d[0]): int(d[1]) for d in data}
    return remove_duplicates(data)

def parse_safire_data(data):
    data = OrderedDict(eval(data))
    t0 = next(iter(data.keys()))
    t0 = int(dp.parse(t0).timestamp())
    data = {int(dp.parse(k).timestamp())-t0:v for k,v in data.items()}
    return remove_duplicates(data)
    

def parse_halucinator_data(data):
    data = eval(data)
    data = {int(k):v for k,v in data.items()}
    data = OrderedDict(data)
    t0 = next(iter(data.keys()))
    data = {k-t0:v for k,v in data.items()}
    return remove_duplicates(data)

# readall data
all_data = {}
for t in targets:
    all_data[t] = {}
    for fw in frameworks:
        print(f"Processing {t} for {fw}")

        all_data[t][fw] = {}
        files = glob.glob(f'{fw}_data/{fw}_{t}*.data')
        assert(len(files) == 5)
        for i in range(5):
            with open(files[i], 'r') as f:
                data = f.read()
            if 'fuzzware' in fw:
                data = parse_fuzzware_data(data)
            elif 'safire' in fw or 'libafl_halucinator' in fw:
                data = parse_safire_data(data)
            elif 'halucinator' in fw:
                data = parse_halucinator_data(data)
            else:
                assert("NOT REACHABLE")
            all_data[t][fw][f'run-{i}'] = data

In [ ]:
for fw in frameworks:
    if fw == 'safire':
        continue
    found_violation = False
    print(f'=== Comparing SafireFuzz with {fw} ===')
    for t in targets:

        #if fw != 'fuzzware_hal':
        #    continue

        safire_maxs = [max(all_data[t]['safire'][f'run-{i}'].values()) for i in range(5) ]
        target_maxs = [max(all_data[t][fw][f'run-{i}'].values()) for i in range(5) ]
        mannu = scipy.stats.mannwhitneyu(safire_maxs, target_maxs, method='auto')
        if mannu.pvalue > 0.05:
            print(f'Offending P-Value while comparing against {t}: {mannu.pvalue:.03f}')
            found_violation=True
    if found_violation is False:
        print('No offending P-Values found')
    print()

In [ ]:
for fw in frameworks:
    print(f'=== Calculating median and stdev for {fw} experiments ===')

    for t in targets:

        target_maxs = [max(all_data[t][fw][f'run-{i}'].values()) for i in range(5) ]
        median = sorted(target_maxs)[2]
        #print(sorted(target_maxs))
        print(f'Sample: {t} - Median: {median} Stddev: {np.std(target_maxs)}')
    print()
            
        